### Build a list with unit dictionaries including below data
- {
- 'musicname':'original filename',
- 'staff_id':int,
- 'start':int,
- 'end':int,
- 'tempo':int,
- 'timesig':'4/4' or '6/8' or...,
- 'beats':'1/4,0; 1/2,1; 1/8,1; 1/8,0' (假設為4/4拍，左為一單位的拍子，順序為：四分休止符、二分音符、八分音符、八分休止符，1/4 + 1/2 + 1/8 + 1/8 = 1)
- }

#### 以下兩格獨立處理{'tempo':int, 'start':int, 'end':int, 'musicname':'original filename'}

In [98]:
from bs4 import BeautifulSoup as bs
import os

#Determine tempo, start measure & end measure
def getTempoMeasure(dirPath, filename, mainList):
    filePath = dirPath + filename
    with open(filePath, 'r') as f:
        mscx = bs(f.read(), 'xml')
    
    #Select measures of staff#1 because tempos only appear in staff#1.
    staff1Measures = mscx.select('Score > Staff:nth-of-type(1) > Measure')
    
    #Get the 1st tempo.
    tempo = staff1Measures[0].select('Tempo > text')[0].text.rsplit(' ', 1)[-1]
    
    #Set tempos, measure numbers, & filename to the list.
    filename = filename.rsplit('.', 1)[0]
    print filename
    tmp = []
    for i, m in enumerate(staff1Measures[:-1], 1):
        for t in m.select('Tempo > text'):
            tmp.append(int(t.text.rsplit(' ', 1)[-1]))
        if i % 4 == 0:
            if tmp:
                tempo = max(tmp)
            mainList.append(genDataDic(tempo, i-3, i, filename))
            tmp[:] = []
    
    #Handle last measure different situations with for-else.
    else:
        i += 1
        for t in staff1Measures[-1].select('Tempo > text'):
            tmp.append(int(t.text.rsplit(' ', 1)[-1]))
        if tmp:
            tempo = max(tmp)    
        if i % 4 == 1: 
            mainList.append(genDataDic(tempo, i, i, filename))
        elif i % 4 == 2:
            mainList.append(genDataDic(tempo, i-1, i, filename))
        elif i % 4 == 3:
            mainList.append(genDataDic(tempo, i-2, i, filename))
        else:
            mainList.append(genDataDic(tempo, i-3, i, filename))

#Generate unit-data dictionary containing below properties
def genDataDic(tempo, start, end, filename):
    return {'tempo':tempo, 'start':start, 'end':end, 'musicname':filename}

In [ ]:
#Determine tempo, start measure, end measure & input the filename
mainList = []
dirPath = 'C:/Users/BigData/Desktop/mscx/'
for filename in os.listdir(dirPath):
    getTempoMeasure(dirPath, filename, mainList)
for e in mainList:
    print e

#### 以下n格獨立處理{'staff_id':int, 'timesig':'4/4', 'beats':'1/4,0; 1/2,1; 1/8,1; 1/8,0'}

In [1]:
from bs4 import BeautifulSoup as bs
from decimal import *
import os

#Beat dictionary.
beatDic = {
           'measure':None,
           'whole':Decimal(1),
           'half':Decimal(1)/2,
           'quarter':Decimal(1)/4,
           'eighth':Decimal(1)/8,
           '16th':Decimal(1)/16,
           '32nd':Decimal(1)/32,
           '64th':Decimal(1)/64,
           '128th':Decimal(1)/128,
           '256th':Decimal(1)/256
          }

In [116]:
#Generate unit-data dictionary containing below properties.
def genDataDic2(staff_id, timeSig, beatStr):
    beatStr = beatStr[:-1]
    return {'staff_id':staff_id, 'timesig':timeSig, 'beats':beatStr}

#Called be function getBeat() to simplify the process handing last measure.
def doLastMeasure(mm, i, sigN, sigD, beatStr):
    timeSig = str(sigN)+'/'+str(sigD)
    if i % 4 == 1:
        mainList2.append(genDataDic2(mm, timeSig, beatStr))
    elif i % 4 == 2:
        mainList2.append(genDataDic2(mm, timeSig, beatStr))
    elif i % 4 == 3:
        mainList2.append(genDataDic2(mm, timeSig, beatStr))
    else:
        mainList2.append(genDataDic2(mm, timeSig, beatStr))
        
#Called by function countBeat() to determine if last note is a Rest.
#whose beat should be merged with this Rest.
def lastRestOrNot(beatStr, beatSum, key, multi):

    #If beatStr is True, it's not the first note(Rest) of a unit.
    if beatStr:
        
        #If last note is a Rest.
        if beatStr[-2] == '0':
            beatSum += beatDic[key] * Decimal(multi)
            preBeat = Decimal(beatStr.rsplit(';', 2)[-2].split(',')[0])
            curBeat = beatDic[key] * Decimal(multi)
            sumBeat = str(preBeat + curBeat)
            
            #beatStr contains equivalent to or more than 2 pairs.
            if len(beatStr.rsplit(';', 2)) > 2:
                beatStr = beatStr.rsplit(';', 2)[-3] + ';' + sumBeat + ',0;'
            
            #beatStr contains exactly 1 pair.
            elif len(beatStr.rsplit(';', 2)) == 2:
                beatStr = sumBeat + ',0;'
        
        #If last note is a Rest.
        else:
            beatSum += beatDic[key] * Decimal(multi)
            beatStr += str(beatDic[key] * Decimal(multi))+',0;'
            
    #If beatStr is False, it's the first note(Rest) of a unit.
    else:
        beatSum += beatDic[key] * Decimal(multi)
        beatStr += str(beatDic[key] * Decimal(multi))+',0;'
    return beatStr, beatSum
    
#Called by function getBeat()
#to count beats & accumulate a beat-string of a unit
def countBeat(measure, beatStr):
    beatSum = 0
    for c in measure.children:
        if str(type(c)) == '<class \'bs4.element.Tag\'>':
            if c.name == 'Chord':
                print c.name
                key = c.find('durationType').text
                if c.find('dots'):
                    dots = int(c.find('dots').text)
                    if dots == 1:
                        beatSum += beatDic[key] * Decimal(1.5)
                        beatStr += str(beatDic[key] * Decimal(1.5))+',1;'
                    elif dots == 2:
                        beatSum += beatDic[key] * Decimal(1.75)
                        beatStr += str(beatDic[key] * Decimal(1.75))+',1;'
                    elif dots == 3:
                        beatSum += beatDic[key] * Decimal(1.875)
                        beatStr += str(beatDic[key] * Decimal(1.875))+',1;'
                else:
                    beatSum += beatDic[key]
                    beatStr += str(beatDic[key])+',1;'
            elif c.name == 'Rest':
                print c.name
                key = c.find('durationType').text
                if c.find('dots'):
                    dots = int(c.find('dots').text)
                    if dots == 1:
                        beatStr, beatSum = lastRestOrNot(beatStr, beatSum, key, 1.5)
                    elif dots == 2:
                        beatStr, beatSum = lastRestOrNot(beatStr, beatSum, key, 1.75)
                    elif dots == 3:
                        beatStr, beatSum = lastRestOrNot(beatStr, beatSum, key, 1.875)
                else:
                    beatStr, beatSum = lastRestOrNot(beatStr, beatSum, key, 1)
    print beatSum
    if beatSum == beatDic['measure']:
        print True
    else:
        print False
    print beatStr
    print '------------------------------------'
    return beatStr

In [117]:
#Determine time signature & beats
#Note: About <tick><track> tags indicating at least
#1 additional melody in a staff, omit them at first.
def getBeat(dirPath, filename, mmList, mainList2):
    filePath = dirPath + filename
    with open(filePath, 'r') as f:
        mscx = bs(f.read(), 'xml')
    
    #Get 1st time signature
    staff1TimeSig = mscx.select_one('Score > Staff:nth-of-type(1) TimeSig')
    sigN = int(staff1TimeSig.find('sigN').text)
    sigD = int(staff1TimeSig.find('sigD').text)
    beatDic['measure'] = Decimal(sigN)/sigD
    diffTimeSigs = False
    
    #Set staff_id, time signature & beats to the list
    for mm in mmList:
        beatStr = ''
        for i, m in enumerate(mscx.select('Score > \
                              Staff:nth-of-type('+str(mm)+') > \
                              Measure')[:-1], 1):
            print '[Staff #'+str(mm)+', Measure #'+str(i)+']'
            
            #Determine Time Signature & detect if it changes
            if m.find('TimeSig'):
                sigN = int(m.find('sigN').text)
                sigD = int(m.find('sigD').text)
                if Decimal(sigN)/sigD != beatDic['measure']:
                    beatDic['measure'] = Decimal(sigN)/sigD
                    if i % 4 == 2 or i % 4 == 3:
                        diffTimeSigs = True
                        continue
                    elif i % 4 == 0:
                        diffTimeSigs = False
                        continue
                        
            #Count beats & accumulate a beat-string of a unit
            beatStr = countBeat(m, beatStr)
            
            #Generate a unit dictionary per 4 measures
            if i % 4 == 0:
                if diffTimeSigs == True:
                    diffTimeSigs = False
                    continue
                timeSig = str(sigN)+'/'+str(sigD)
                mainList2.append(genDataDic2(mm, timeSig, beatStr))
                beatStr = ''
                
        #Handle last-measure situations with for-ELSE.
        else:
            i += 1
            lastMeasure = mscx.select('Score > \
                                       Staff:nth-of-type('+str(mm)+') > \
                                       Measure')[-1]
            print '[Staff #'+str(mm)+', Measure #'+str(i)+']'
            if lastMeasure.find('TimeSig'):
                sigN = int(lastMeasure.find('sigN').text)
                sigD = int(lastMeasure.find('sigD').text)
                if Decimal(sigN)/sigD != beatDic['measure']:
                    break
                else:
                    beatStr = countBeat(lastMeasure, beatStr)
                    doLastMeasure(mm, i, sigN, sigD, beatStr)
            beatStr = countBeat(lastMeasure, beatStr)
            doLastMeasure(mm, i, sigN, sigD, beatStr)

In [119]:
#Numbers of staffs which play main melodies
mmList = [4]

#Determine tempo, start measure, end measure & input the filename
#Determine time signature, beats, staff-d,
mainList2 = []
dirPath = 'C:/Users/BigData/Desktop/mscx/'
for filename in os.listdir(dirPath):
    getBeat(dirPath, filename, mmList, mainList2)
for j, e in enumerate(mainList2, 1):
    print '[Unit #'+str(j)+']'
    print e
    print '------------------------------------'

[Staff #4, Measure #1]
Rest
1
True
1,0;
------------------------------------
[Staff #4, Measure #2]
Rest
1
True
2,0;
------------------------------------
[Staff #4, Measure #3]
Rest
1
True
3,0;
------------------------------------
[Staff #4, Measure #4]
Rest
1
True
4,0;
------------------------------------
[Staff #4, Measure #5]
Rest
1
True
1,0;
------------------------------------
[Staff #4, Measure #6]
Rest
1
True
2,0;
------------------------------------
[Staff #4, Measure #7]
Rest
1
True
3,0;
------------------------------------
[Staff #4, Measure #8]
Rest
1
True
4,0;
------------------------------------
[Staff #4, Measure #9]
Rest
1
True
1,0;
------------------------------------
[Staff #4, Measure #10]
Rest
1
True
2,0;
------------------------------------
[Staff #4, Measure #11]
Rest
1
True
3,0;
------------------------------------
[Staff #4, Measure #12]
Rest
1
True
4,0;
------------------------------------
[Staff #4, Measure #13]
Rest
1
True
1,0;
--------------------------------

In [ ]:
for i in range(0,len(music.select('Chord'))):
    if len(music.select('Chord')[i].select('Tie')):
        key1 = music.select('Chord')[i].select('durationType')[0].text
        key2 = music.select('Chord')[i+1].select('durationType')[0].text
        beatSum = beatDic[key1]+beatDic[key2]
        print 'tie id = '+ music.select('Chord')[i].select('Tie')[0].get('id')
        print beatSum